In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

import tempfile
import random
import sys
root = '../../../caffe'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, root + 'python')

import caffe
import os
from sklearn import linear_model
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

caffe.set_device(0)
caffe.set_mode_gpu()

In [2]:
import skimage.exposure as exposure
import skimage.transform as transform
import skimage.io as io
from time import time
from time import sleep

In [3]:
seed = random.seed(100)

In [4]:
alexpath = '../../../networks/alexnet/' 
vggpath = '../../../networks/vgg/'
images = '../../../data/food-101/images/'
meanimage = np.load('../../../caffe/python/caffe/imagenet/ilsvrc_2012_mean.npy')
meanimage = meanimage.mean(1).mean(1)

In [5]:
if os.path.isfile(alexpath + "alexnet.caffemodel"):
    print 'alex'
if os.path.isfile(vggpath + "VGG_ILSVRC_16_layers.caffemodel"):
    print 'vgg'

alex
vgg


In [6]:
with open('../../../data/food-101/meta/train.json') as data_file:    
    train = json.load(data_file)

with open('../../../data/food-101/meta/test.json') as data_file:    
    test = json.load(data_file) 
    
classes = list(np.loadtxt('../../../projects/irina.tirosyan/1.CNN-Classification/classes.txt', str, delimiter='\n'))

In [7]:
#ten = random.sample(classes, 10)
#thirty = random.sample(classes, 30)

In [8]:
vgg_def = '../../../networks/vgg/deploy.prototxt'
vgg_weights  = '../../../networks/vgg/VGG_ILSVRC_16_layers.caffemodel'
vgg = caffe.Net(vgg_def, vgg_weights, caffe.TEST)
#vgg.blobs['data'].reshape(50, 3, 224, 224)
vggtransformer = caffe.io.Transformer({'data': vgg.blobs['data'].data.shape})
vggtransformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
vggtransformer.set_mean('data', meanimage)            # subtract the dataset-mean value in each channel
vggtransformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
vggtransformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [9]:
alex_def = '../../../networks/alexnet/deploy.prototxt'
alex_weights  = '../../../networks/alexnet/alexnet.caffemodel'
alex = caffe.Net(alex_def, alex_weights, caffe.TEST)
#alex.blobs['data'].reshape(50, 3, 227, 227)
alextransformer = caffe.io.Transformer({'data': alex.blobs['data'].data.shape})
alextransformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
alextransformer.set_mean('data', meanimage)            # subtract the dataset-mean value in each channel
alextransformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
alextransformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [10]:
def standardize(image, size):
    existing_shape = image.shape
    if existing_shape[1] > existing_shape[0]:
        image = transform.rotate(image, 90, resize=True)
    if image.shape != size:
        image = transform.resize(image, size)
    return image

In [11]:
def features(img, subset, layer, net):    
    image = caffe.io.load_image(home + '../../../data/food-101/images/' + img[subset] + '.jpg')
    transformed = transformer.preprocess('data', image)
    net.blobs['data'].data[...] = transformed
    net.forward()
    return list(net.blobs[layer].data[0])

In [12]:
tenl = random.sample(train.keys(), 10)
thirtyl = random.sample(train.keys(), 30)

In [13]:
def getdata(classes, data):
    x, y = [], []
    for i in range(len(classes)):
        key = classes[i]
        x  = x + data[key]
        y = y + [i]*len(data[key])
    return x,y

In [14]:
traintenx, trainteny = getdata(tenl, train)
testtenx, testteny = getdata(tenl, test)
trainthirtyx, trainthirtyy = getdata(thirtyl, train)
testthirtyx, testthirtyy = getdata(thirtyl, test)

In [15]:
#http://stackoverflow.com/questions/32931280/extract-image-features-using-caffe-for-custom-image-classifier

def extractfeaturesvgg(imgs, batch, layer):
    features = np.empty((len(imgs),4096))
    nxt = 0
    for i in range(len(imgs)/batch):
        images = []
        wholebatch = imgs[batch*i : batch*(i+1)] 
        for j in wholebatch:
            img = caffe.io.load_image('../../../data/food-101/images/' + str(j) + '.jpg')
            transformed_image = vggtransformer.preprocess('data', img)
            images.append(transformed_image)
        vgg.blobs['data'].data[...] = images
        vgg.forward()
        for f in vgg.blobs[layer].data:
            features[nxt] =  f
            nxt = nxt + 1
    return features

def extractfeaturesalex(imgs, batch, layer):
    features = np.empty((len(imgs),4096))
    nxt = 0
    for i in range(len(imgs)/batch):
        images = []
        wholebatch = imgs[batch*i : batch*(i+1)]   
        for j in wholebatch:
            img = caffe.io.load_image('../../../data/food-101/images/' + str(j) + '.jpg')
            transformed_image = alextransformer.preprocess('data', img)
            images.append(transformed_image)
        alex.blobs['data'].data[...] = images
        alex.forward()
        for f in alex.blobs[layer].data:
            features[nxt] =  f
            nxt = nxt + 1
    return features

# VGGx30

In [16]:
t0 = time()
trainthirtyf = extractfeaturesvgg(trainthirtyx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

2283.82 seconds


In [17]:
t0 = time()
testthirtyf = extractfeaturesvgg(testthirtyx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

761.73 seconds


In [18]:
clf_vgg30 = linear_model.SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
            eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
            learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
            penalty='l2', power_t=0.5, random_state=None, shuffle=True,
            verbose=0, warm_start=False)

In [19]:
clf_vgg30.fit(trainthirtyf, np.asarray(trainthirtyy))

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [20]:
clf_vgg30.score(trainthirtyf, np.asarray(trainthirtyy))

0.95022222222222219

In [63]:
ac = clf_vgg30.score(testthirtyf, np.asarray(testthirtyy))

In [54]:
f = open('vgg30accuracy.txt', 'a') 
f.write(str(ac))
f.write('\n') 
f.close() 

In [96]:
vgg30accuracy = list(np.loadtxt('vgg30accuracy.txt', float, delimiter='\n'))

In [97]:
print 'Average vgg30 accuracy is:' 
print np.mean(vgg30accuracy)

Average vgg30 accuracy is:
0.752534482957


In [22]:
np.savez_compressed('../../../projects/irina.tirosyan/1.CNN-Classification/vgg30', vgg30_train = trainthirtyf, vgg30_test = testthirtyf)

In [23]:
print(classification_report(testthirtyy, clf_vgg30.predict(testthirtyf), target_names = thirtyl))

                      precision    recall  f1-score   support

        spring_rolls       0.61      0.74      0.67       250
          cheesecake       0.49      0.57      0.53       250
              nachos       0.80      0.45      0.57       250
           escargots       0.64      0.82      0.72       250
       caprese_salad       0.66      0.74      0.70       250
           hamburger       0.74      0.59      0.65       250
              paella       0.80      0.60      0.68       250
          crab_cakes       0.70      0.34      0.46       250
           apple_pie       0.69      0.41      0.51       250
        creme_brulee       0.82      0.80      0.81       250
        clam_chowder       0.87      0.84      0.86       250
       seaweed_salad       0.86      0.81      0.83       250
             risotto       0.67      0.57      0.62       250
            macarons       0.88      0.94      0.91       250
        deviled_eggs       0.61      0.83      0.71       250
       

# Alexnetx30

In [24]:
t0 = time()
trainthirtyf_alex = extractfeaturesalex(trainthirtyx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

657.00 seconds


In [25]:
t0 = time()
testthirtyf_alex = extractfeaturesalex(testthirtyx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

223.58 seconds


In [26]:
clf_alex30 = linear_model.SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
            eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
            learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
            penalty='l2', power_t=0.5, random_state=None, shuffle=True,
            verbose=0, warm_start=False)

In [27]:
clf_alex30.fit(trainthirtyf_alex, np.asarray(trainthirtyy))

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [28]:
clf_alex30.score(trainthirtyf_alex, np.asarray(trainthirtyy))

0.92755555555555558

In [29]:
clf_alex30.score(testthirtyf_alex, np.asarray(testthirtyy))

0.6041333333333333

In [72]:
ac = clf_alex30.score(testthirtyf_alex, np.asarray(testthirtyy))
ac

0.6041333333333333

In [74]:
f = open('alex30accuracy.txt', 'a') 
f.write(str(ac))
f.write('\n') 
f.close() 

In [98]:
alex30accuracy = list(np.loadtxt('alex30accuracy.txt', float, delimiter='\n'))

In [99]:
print 'Average alex30 accuracy is:' 
print np.mean(alex30accuracy)

Average alex30 accuracy is:
0.656716137602


In [30]:
np.savez_compressed('../../../projects/irina.tirosyan/1.CNN-Classification/alex30', alex30_train = trainthirtyf_alex, alex30_test = testthirtyf_alex)

In [31]:
print(classification_report(testthirtyy, clf_alex30.predict(testthirtyf_alex), target_names = thirtyl))

                      precision    recall  f1-score   support

        spring_rolls       0.57      0.57      0.57       250
          cheesecake       0.62      0.45      0.52       250
              nachos       0.37      0.62      0.46       250
           escargots       0.85      0.51      0.64       250
       caprese_salad       0.46      0.65      0.54       250
           hamburger       0.53      0.45      0.49       250
              paella       0.44      0.67      0.53       250
          crab_cakes       0.48      0.40      0.43       250
           apple_pie       0.45      0.48      0.46       250
        creme_brulee       0.71      0.75      0.73       250
        clam_chowder       0.76      0.81      0.78       250
       seaweed_salad       0.76      0.82      0.79       250
             risotto       0.53      0.42      0.47       250
            macarons       0.88      0.83      0.85       250
        deviled_eggs       0.75      0.66      0.70       250
       

# VGGx10

In [32]:
t0 = time()
traintenf = extractfeaturesvgg(traintenx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

770.45 seconds


In [33]:
t0 = time()
testtenf = extractfeaturesvgg(testtenx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

259.78 seconds


In [34]:
clf_vgg10 = linear_model.SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
            eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
            learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
            penalty='l2', power_t=0.5, random_state=None, shuffle=True,
            verbose=0, warm_start=False)

In [35]:
clf_vgg10.fit(traintenf, np.asarray(trainteny))

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [36]:
clf_vgg10.score(traintenf, np.asarray(trainteny))

0.98640000000000005

In [37]:
clf_vgg10.score(testtenf, np.asarray(testteny))

0.78439999999999999

In [80]:
ac = clf_vgg10.score(testtenf, np.asarray(testteny))

In [82]:
f = open('vgg10accuracy.txt', 'a') 
f.write(str(ac))
f.write('\n') 
f.close() 

In [100]:
vgg10accuracy = list(np.loadtxt('vgg10accuracy.txt', float, delimiter='\n'))

In [101]:
print 'Average vgg10 accuracy is:' 
print np.mean(vgg10accuracy)

Average vgg10 accuracy is:
0.83471997479


In [38]:
np.savez_compressed('../../../projects/irina.tirosyan/1.CNN-Classification/vgg10', vgg10_train = traintenf, vgg10_test = testtenf)

In [39]:
print(classification_report(testteny, clf_vgg10.predict(testtenf), target_names = tenl))

                  precision    recall  f1-score   support

        bibimbap       0.91      0.90      0.91       250
 red_velvet_cake       0.94      0.93      0.94       250
    cheese_plate       0.76      0.72      0.74       250
shrimp_and_grits       0.83      0.83      0.83       250
       hamburger       0.82      0.86      0.84       250
      beet_salad       0.75      0.77      0.76       250
        takoyaki       0.80      0.81      0.81       250
     peking_duck       0.77      0.74      0.75       250
       foie_gras       0.61      0.55      0.58       250
  grilled_salmon       0.65      0.74      0.69       250

     avg / total       0.78      0.78      0.78      2500



# Alexnetx10

In [40]:
t0 = time()
traintenf_alex = extractfeaturesalex(traintenx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

232.86 seconds


In [41]:
t0 = time()
testtenf_alex = extractfeaturesalex(testtenx, 10, 'fc7')
print("%0.2f seconds" % (time() - t0))

75.00 seconds


In [42]:
clf_alex10 = linear_model.SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
            eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
            learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
            penalty='l2', power_t=0.5, random_state=None, shuffle=True,
            verbose=0, warm_start=False)

In [43]:
clf_alex10.fit(traintenf_alex, np.asarray(trainteny))

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=20, n_jobs=4,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [44]:
clf_alex10.score(traintenf_alex, np.asarray(trainteny))

0.99253333333333338

In [45]:
clf_alex10.score(testtenf_alex, np.asarray(testteny))

0.71960000000000002

In [89]:
ac = clf_alex10.score(testtenf_alex, np.asarray(testteny))

In [90]:
f = open('alex10accuracy.txt', 'a') 
f.write(str(ac))
f.write('\n') 
f.close() 

In [102]:
alex10accuracy = list(np.loadtxt('alex10accuracy.txt', float, delimiter='\n'))

In [103]:
print 'Average alex10 accuracy is:' 
print np.mean(alex10accuracy)

Average alex10 accuracy is:
0.77551179461


In [46]:
np.savez_compressed('../../../projects/irina.tirosyan/1.CNN-Classification/alex10', alex10_train = traintenf_alex, alex10_test = testtenf_alex)

In [47]:
print(classification_report(testteny, clf_alex10.predict(testtenf_alex), target_names = tenl))

                  precision    recall  f1-score   support

        bibimbap       0.89      0.87      0.88       250
 red_velvet_cake       0.78      0.89      0.83       250
    cheese_plate       0.72      0.55      0.62       250
shrimp_and_grits       0.72      0.84      0.78       250
       hamburger       0.70      0.78      0.74       250
      beet_salad       0.71      0.70      0.70       250
        takoyaki       0.82      0.71      0.76       250
     peking_duck       0.69      0.69      0.69       250
       foie_gras       0.57      0.61      0.59       250
  grilled_salmon       0.59      0.56      0.58       250

     avg / total       0.72      0.72      0.72      2500

